## Prep:
1. Make a Twitter account
2. Sign up at https://dev.twitter.com for a developer account
    a. Ensure it has read/write access
3. Make an new app under your development account
4. Generate an access token key pair
5. Rename `twitter_keys_sample.json` to `twitter_keys.json`
6. Copy tokens to their respective fields in `twitter_keys.json`
7. Install tweepy

Import the libraries we need for this script
* json - used for parsing JSON files
* datetime - used to calculate yesterday's date
* requests - makes HTTP requests. This will be used to make our API calls to the WPRDC.
* tweepy - used to make Twitter API calls.

In [1]:
import json
from datetime import datetime, timedelta
import requests
import tweepy

The URL of the API call we want to make.  
`action/datastore_search_sql` searches a resource (data table) on the WPRDC using a supplied SQL statement

In [3]:
wprdc_api_endpoint = "https://data.wprdc.org/api/3/action/datastore_search_sql"

The resource ID of the data we want to collect. In this case, it's the `Blotter Data` resource in the [Police Blotter (30 day)](https://data.wprdc.org/dataset/police-incident-blotter) dataset.

In [8]:
resource_id = "1797ead8-8262-41cc-9099-cbc8a161924b"

Generate yesterday's date to use in our SQL query to the WPRDC.

In [16]:
# Get yesterday's date (the current date - 1 day)
yesterday_date = datetime.now() - timedelta(days=1)
yesterday_date

'2016-11-04'

In [ ]:
# Convert to a string format that the Data Center accepts (yyyy-mm-dd)
yesterday_str = yesterday_date.strftime("%Y-%m-%d")
yesterday_str

Generate a SQL query to select all records from the `Blotter Data` where the date is greater than or equal to yesterday's date.

In [19]:
query = "SELECT count(\"PK\") as count FROM \"{}\" WHERE \"INCIDENTTIME\" >= '{}';".format(resource_id, yesterday_str)
query

'SELECT count("PK") as count FROM "1797ead8-8262-41cc-9099-cbc8a161924b" WHERE "INCIDENTTIME" >= \'2016-11-04\';'

Make a call to the WPRDC API
We'll be using [`datastore_search_sql`](http://docs.ckan.org/en/latest/maintaining/datastore.html#ckanext.datastore.logic.action.datastore_search_sql) which takes only one parameter (argument): `sql`, a SQL query string.
It will return a JSON response in the following format:
```json
{
  "help": "https://data.wprdc.org/api/3/action/help_show?name=datastore_search_sql",
  "success": true,
  "result": {
    "records": [
      {
        "record1_field1": "a value",
        "record1_field2": "another value"
      },
      {
        "record2_field1": "some value",
        "record2_field2": "some other value"
      }
      ...  // this keeps going for each record (row) returned from your query
    ],
    "fields": [
      {
        "type": "field1's type",
        "id": "field1's name"
      },
      {
        "type": "field2's type",
        "id": "field2's name"
    ],
    "sql": "The SQL statment you sent in your request"
  }
}
```

What we're interested in is what's in result => records.
It's a list of objects which works out to be a table. Each item in the list is a row, while each object is a mapping of a column name to its value in that row.

Make API call using requests

In [26]:
response = requests.get(wprdc_api_endpoint, {'sql': query})
response

<Response [200]>

printing the `response` object only shows its [status code](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
However, the `response` object has a `text` attribute which contains the JSON-formated response we mentioned above.

In [27]:
response.text

'{"help": "https://data.wprdc.org/api/3/action/help_show?name=datastore_search_sql", "success": true, "result": {"records": [{"count": "81"}], "fields": [{"type": "int8", "id": "count"}], "sql": "SELECT count(\\"PK\\") as count FROM \\"1797ead8-8262-41cc-9099-cbc8a161924b\\" WHERE \\"INCIDENTTIME\\" >= \'2016-11-04\';"}}'

In [28]:
response_data = json.loads(response.text)
response_data

{'help': 'https://data.wprdc.org/api/3/action/help_show?name=datastore_search_sql',
 'result': {'fields': [{'id': 'count', 'type': 'int8'}],
  'records': [{'count': '81'}],
  'sql': 'SELECT count("PK") as count FROM "1797ead8-8262-41cc-9099-cbc8a161924b" WHERE "INCIDENTTIME" >= \'2016-11-04\';'},
 'success': True}

In [34]:
print(json.dumps(response_data, sort_keys=True, indent=2, separators=(',', ': ')))  # just to demonstrate the JSON format - not required

{
  "help": "https://data.wprdc.org/api/3/action/help_show?name=datastore_search_sql",
  "result": {
    "fields": [
      {
        "id": "count",
        "type": "int8"
      }
    ],
    "records": [
      {
        "count": "81"
      }
    ],
    "sql": "SELECT count(\"PK\") as count FROM \"1797ead8-8262-41cc-9099-cbc8a161924b\" WHERE \"INCIDENTTIME\" >= '2016-11-04';"
  },
  "success": true
}


In [33]:
count = response_data['result']['records'][0]['count']
count

'81'

The count is returned to us a string. If we plan to do any math with it, we'd have to convert it to an int, but since we're just printing it, we can leave it as is.

That's it for the WPRDC request; now it's time to use the count we retrieved in a tweet.

# Twitter Section

First we need to load the Twitter keys into variables.  
*We're reading ours from a file, so we can use real keys without publicly displaying them. You can simply put them into your source code if you don't plan on making it publicly available.*

If you want to use a separete key file like we do, this is what the provided sample file looks like.  Simply replace the values (to the right of the `:`) with your keys.

In [36]:
with open('twitter_keys_sample.json') as f:
    twitter_keys = json.load(f)
    
print(json.dumps(twitter_keys, sort_keys=True, indent = 2, separators=(',', ': ')))

{
  "access_token_key": "acCe5s70kEn",
  "access_token_secret": "acCe5s70kEn53crEt",
  "consumer_key": "c0n5uM3rk3Y",
  "consumer_secret": "c0N5um3r53cR3t"
}


In [37]:
with open('twitter_keys.json') as f:
    twitter_keys = json.load(f)

In [38]:
consumer_key = twitter_keys['consumer_key']
consumer_secret = twitter_keys['consumer_secret']
access_token_key = twitter_keys['access_token_key']
access_token_secret = twitter_keys['access_token_secret']

Now we need to let Twitter know we are who we say we are by using our keys and secrets

In [39]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)

This gives us an object that proves we have authorized properly and can use Twitter under our account. We use that as the argument for tweepy.API which makes API calls to Twitter.

In [45]:
twitter = tweepy.API(auth)
twitter

Now we can use our `twitter` object to send Tweets

In [46]:
twitter.update_status('Gee willickers! There were {} crime incidents in Pittsburgh yesterday.'.format(count))

Status(contributors=None, in_reply_to_status_id_str=None, coordinates=None, _api=<tweepy.api.API object at 0x0000016B6B061940>, user=User(listed_count=0, protected=False, profile_background_image_url=None, lang='en', favourites_count=0, profile_image_url_https='https://abs.twimg.com/sticky/default_profile_images/default_profile_4_normal.png', description='', time_zone=None, id_str='793281895622553600', id=793281895622553600, is_translation_enabled=False, statuses_count=2, _json={'profile_background_tile': False, 'protected': False, 'profile_background_image_url': None, 'notifications': False, 'description': '', 'favourites_count': 0, 'following': False, 'lang': 'en', 'time_zone': None, 'id_str': '793281895622553600', 'profile_sidebar_fill_color': 'DDEEF6', 'id': 793281895622553600, 'is_translation_enabled': False, 'has_extended_profile': False, 'profile_text_color': '333333', 'profile_image_url_https': 'https://abs.twimg.com/sticky/default_profile_images/default_profile_4_normal.png', 